In [50]:
import pandas as pd
import matplotlib.pyplot  as plt
import os
import librosa
import numpy as np
import scipy
import keras
import torch

Метки классов

гнев - 0

грусть - 1

радость - 2

страх - 3

Отбор признаков

In [51]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
dir = "/content/drive/MyDrive/"

In [54]:
dirname_anger = os.path.join(dir, 'data', 'гнев')
audio_anger = [os.path.join(dirname_anger, file) for file in os.listdir(dirname_anger) if file.endswith('.wav')]

dirname_sad = os.path.join(dir, 'data', 'грусть')
audio_sad = [os.path.join(dirname_sad, file) for file in os.listdir(dirname_sad) if file.endswith('.wav')]

dirname_happy = os.path.join(dir, 'data', 'радость')
audio_happy = [os.path.join(dirname_happy, file) for file in os.listdir(dirname_happy) if file.endswith('.wav')]

dirname_fear = os.path.join(dir, 'data', 'страх')
audio_fear = [os.path.join(dirname_fear, file) for file in os.listdir(dirname_fear) if file.endswith('.wav')]


In [55]:
import librosa
import pandas as pd

# Создание пустого DataFrame для хранения признаков
df = pd.DataFrame()

# Список аудио файлов
audio_files = audio_anger + audio_sad + audio_happy + audio_fear

# Извлечение признаков из каждого аудио файла
for file in audio_files:
    # Загрузка аудио файла
    audio, sr = librosa.load(file)

    # Извлечение признаков
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=12)  # MFCC признаки
    spectral_centroid = librosa.feature.spectral_centroid(y=audio, sr=sr)  # Центроид спектра
    fundamental_freq = librosa.yin(y=audio, fmin=50, fmax=400)  # Основная частота

    # Определение метки класса на основе названия аудио файла
    label = None
    if 'гнев' in file:
        label = "0"
    elif 'грусть' in file:
        label = "1"
    elif 'радость' in file:
        label = "2"
    elif 'страх' in file:
        label = "3"

    # Создание словаря с извлеченными признаками и меткой класса
    extracted_features = {
        'meanfreq': np.mean(mfcc[2]),  # Средняя частота
        'sd': np.std(mfcc[3]),  # Стандартное отклонение
        'median': np.median(mfcc[4]),  # Медиана
        'Q25': np.percentile(mfcc[5], 25),  # 25-й перцентиль (нижний квартиль)
        'Q75': np.percentile(mfcc[6], 75),  # 75-й перцентиль (верхний квартиль)
        'IQR': np.percentile(mfcc[7], 75) - np.percentile(mfcc[7], 25),  # Межквартильный размах
        'sp.ent': spectral_centroid[0].mean(),  # Энтропия спектра
        'sfm': np.mean(fundamental_freq),  # Спектральный плоскостной максимум
        'label': label  # Метка класса
    }

    # Добавление признаков в DataFrame
    df = df.append(extracted_features, ignore_index=True)

df.index = [os.path.basename(file) for file in audio_files]

<ipython-input-55-eb138907d70e>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_features, ignore_index=True)
<ipython-input-55-eb138907d70e>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_features, ignore_index=True)
<ipython-input-55-eb138907d70e>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_features, ignore_index=True)
<ipython-input-55-eb138907d70e>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(extracted_features, ignore_index=True)
<ipython-input-55-eb138907d70e>:50: FutureWarning: The frame.append method is deprecated and will be

In [ ]:
df.isnull().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
label       0
dtype: int64

In [56]:
y = df["label"]
del df["label"]

In [57]:
df = df.reset_index(drop=True)

In [58]:
df

,meanfreq,sd,median,Q25,Q75,IQR,sp.ent,sfm
0,-31.721300,14.565228,-1.657457,3.785291,-7.913050,13.675529,3035.121202,214.862460
1,-25.408955,18.991066,-14.694403,-19.089689,-2.629512,18.337553,2714.158773,266.248193
2,-61.242405,20.478550,18.929230,-17.586204,13.060294,13.550378,1659.998061,120.465022
3,-5.549602,21.636679,7.665258,-13.449239,-17.151848,16.521714,2452.521724,221.871374
4,-25.721161,17.824408,-8.455436,-8.374517,-1.633892,21.488476,2633.508224,246.069567
...,...,...,...,...,...,...,...,...
123,-14.678594,23.490438,-10.549891,1.837524,-4.213425,8.627972,2314.777446,199.006901
124,0.769895,14.282634,-26.121471,6.586935,-8.023740,14.681534,3230.710241,258.974616
125,-17.793982,15.432932,-11.706542,2.965881,6.333898,12.647416,1659.125590,159.592281
126,-15.694260,23.261852,-10.461660,-3.200342,3.226359,20.434905,2384.060632,228.796127


In [59]:
X = df

In [60]:
X.shape

(128, 8)

In [ ]:
y.shape

(128,)

In [61]:
from sklearn.model_selection import train_test_split


X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
X_train

,meanfreq,sd,median,Q25,Q75,IQR,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun
94,-25.731697,20.442421,13.069578,-25.264805,-7.661883,17.203104,1972.719849,286.956879,-38.793484,1972.719849,-1.575566,-35.575058,21.989569
30,3.560168,18.272667,-6.500600,0.808250,3.519719,11.008638,2512.332275,237.336945,-27.400841,2512.332275,-0.019643,-27.261856,9.980785
33,-45.848682,21.897287,-51.103706,5.719282,-17.312075,14.093142,2925.876465,255.805099,0.000000,2925.876465,-11.303429,-23.878641,37.046013
2,-61.242405,20.478550,18.929230,-17.586205,13.060294,13.550379,1659.998047,120.465019,-4.098671,1659.998047,7.167807,-26.904263,40.458607
59,21.130379,23.936245,18.877756,0.355121,14.005813,11.656671,778.013184,56.252613,-5.553617,778.013184,6.998698,-16.739038,21.024706
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,-108.248604,10.679961,-38.112095,-25.129475,-4.268407,16.371853,2095.724609,136.025925,-39.539299,2095.724609,-1.242718,-9.166284,35.632706
64,-7.279070,19.527161,5.597650,-14.807472,15.400532,22.393269,1427.069214,201.732132,-35.959579,1427.069214,3.890634,-14.523674,25.157143
117,-5.232584,21.004576,-22.588902,-10.710368,5.557233,17.931271,2846.747314,203.354492,-18.744431,2846.747314,5.978992,-29.508900,24.267292
47,3.996482,18.246534,18.024096,-0.520676,5.999732,12.627396,2482.289062,113.433815,-52.153915,2482.289062,2.113179,-17.188959,18.517302


In [ ]:
X_train.shape

(115, 13)

In [ ]:
X_test.shape

(20, 10)

In [ ]:
y_train.shape

(108,)

In [ ]:
y_test.shape

(20,)

In [63]:
import torch.utils.data as torch_data

X_train_array = X_train.values.astype(np.float32)
y_train_array = y_train.values.astype(np.long)
X_test_array = X_test.values.astype(np.float32)
y_test_array = y_test.values.astype(np.long)

# Преобразование массивов в тензоры PyTorch
X_train_tensor = torch.FloatTensor(X_train_array)
y_train_tensor = torch.LongTensor(y_train_array)
X_test_tensor = torch.FloatTensor(X_test_array)
y_test_tensor = torch.LongTensor(y_test_array)

# Создание DataLoader для обучающих и тестовых данных
batch_size = 12
train_dataset = torch_data.TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = torch_data.TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = torch_data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch_data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

<ipython-input-63-173b9029771a>:4: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_array = y_train.values.astype(np.long)
<ipython-input-63-173b9029771a>:6: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which 

In [64]:
import torch.nn.functional as F
import torch
import torch.nn as nn


class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=11, hidden_size=8, num_layers=1, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=8, hidden_size=6, num_layers=1, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=6, hidden_size=4, num_layers=1, batch_first=True)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x = F.relu(x)
        x, _ = self.lstm2(x)
        x = F.relu(x)
        x, _ = self.lstm3(x)
        x = F.softmax(x)
        return x

In [65]:
loss_fn = nn.CrossEntropyLoss()

In [66]:
from sklearn.metrics import accuracy_score

def train_model(model, loader, loss_fn, optimizer, n_epochs):
    model.train()

    train_losses = []
    test_losses = []

    for epoch in range(1, n_epochs + 1):
        for X_batch, y_batch in loader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0:
            model.eval()
            with torch.no_grad():
                y_train_pred = model(X_train_tensor)
                train_acc = accuracy_score(y_train_tensor, torch.argmax(y_train_pred, dim=1))
                print(f"Epoch {epoch}: Train Accuracy: {train_acc:.3f}")


In [67]:
import torch.optim as optim

model = LSTMModel()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [68]:
n_epochs = 20
train_model(model, train_dataloader, loss_fn, optimizer, n_epochs)

<ipython-input-64-2feb9d9c6b14>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


Epoch 10: Train Accuracy: 0.265


<ipython-input-64-2feb9d9c6b14>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


Epoch 20: Train Accuracy: 0.265


In [69]:
torch.save(model, f"./lstm_forecast_{n_epochs}_epochs.pth")

In [76]:
def calculate_accuracy(y_pred, y_true):
    _, predicted = torch.max(y_pred, 0)
    correct = (predicted == y_true).sum().item()
    total = y_true.size(0)
    accuracy = correct / total
    return accuracy

def get_predictions(model, dataloader):
    model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, dim=1)
            predictions.extend(predicted.tolist())
            true_labels.extend(labels.tolist())

    return predictions, true_labels

test_predictions, true_labels = get_predictions(model, test_dataloader)

accuracy = calculate_accuracy(torch.tensor(test_predictions), torch.tensor(true_labels))
print(f'Test Accuracy: {accuracy}')

Test Accuracy: 0.3076923076923077


<ipython-input-64-2feb9d9c6b14>:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)
